# __SARIMAX__

## __Seasonal AutoRegressive Integrated Moving Average with eXogenous regressors__

So far our predicitions were based on past values and errors of our dataset in order to predict trends, seasonality, and forecasted values. The following model encompasses not only non-seasonal (p, d, q) and seasonal (P, D, Q, m) factors, but more enable the inclusion of exogenous variables (e.g. environmental).

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

from statsmodels.tsa.statespace.sarimax import SARIMAX

from statsmodels.graphics.tsaplots import plot_acf,plot_pacf # for determining (p,q) orders manually
from statsmodels.tsa.seasonal import seasonal_decompose      # for ETS plots (seasonal_decompose)
from pmdarima import auto_arima                              # for determining ARIMA orders

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

### __Inspect the data__
The data considers daily visitors to four restaurants located in the United States, subject to American holidays. For the exogenous variable we'll see how holidays affect patronage. The dataset contains 478 days of restaurant data, plus an additional 39 days of holiday data for forecasting purposes.

In [ ]:
df = pd.read_csv('../data/RestaurantVisitors.csv',index_col='date',parse_dates=True)
df.index.freq = 'D'

In [ ]:
df.head()

Notice that even though the restaurant visitor columns contain integer data, they appear as floats. This is because the bottom of the dataframe has 39 rows of NaN data to accommodate the extra holiday data we'll use for forecasting, and pandas won't allow NaN's as integers. We could leave it like this, but since we have to drop NaN values anyway, let's also convert the columns to dtype int64.

In [ ]:
df.tail()

In [ ]:
df1 = df.dropna()
df1.tail()

In [ ]:
cols = ['rest1','rest2','rest3','rest4','total']
for col in cols:
    df1[col] = df1[col].astype(int)
df1.head()

### __Plot of source data__

In [ ]:
plt.style.use('ggplot')

title='Restaurant Visitors'
ylabel='Visitors per day' #do not need an xlabeö

ax = df1['total'].plot(figsize=(16,5),title=title, color='blue')
ax.autoscale(axis='x',tight=True)
ax.set(ylabel=ylabel);

## __Look at holidays as an external feature__
Rather than prepare a separate plot, we can use matplotlib to shade holidays behind our restaurant data.

In [ ]:
plt.style.use('ggplot')

title='Restaurant Visitors'
ylabel='Visitors per day' #do not need xlabel

ax = df1['total'].plot(figsize=(16,5),title=title, color='blue')
ax.autoscale(axis='x',tight=True)
ax.set(ylabel=ylabel)
for x in df1[df1['holiday'] == 1].index:    
    ax.axvline(x=x, color='k', alpha = 0.3);  #adds semi-transparent grey line

### __Run seasonal_decompose__

In [ ]:
result = seasonal_decompose(df1['total'])
result.plot();

## __Test for stationarity__

In [ ]:
from statsmodels.tsa.stattools import adfuller

def adf_test(series,title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(),autolag='AIC') # .dropna() handles differenced data
    
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4],index=labels)

    for key,val in result[4].items():
        out[f'critical value ({key})']=val
        
    print(out.to_string())          # .to_string() removes the line "dtype: float64"
    
    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis")
        print("Reject the null hypothesis")
        print("Data has no unit root and is stationary")
    else:
        print("Weak evidence against the null hypothesis")
        print("Fail to reject the null hypothesis")
        print("Data has a unit root and is non-stationary")

In [ ]:
adf_test(df1['total'])

### __Run pmdarima.auto_arima to get recommended orders__
That may take a while as different combinations of orders are calculated here to compare different AICs. Again, optimal order combinations may vary with computer power.

In [ ]:
# For SARIMA Orders we set seasonal=True and pass in an m value
auto_arima(df1['total'],seasonal=True,m=7).summary()


This provides an ARIMA Order of (1,0,0) and a seasonal order of (2,0,0,7) Now let's train & test the SARIMA model, evaluate it, then compare the result to a model that uses an exogenous variable.

### __train / test split__
We'll assign 42 days (6 weeks) to the test set so that it includes several holidays.

In [ ]:
len(df1)

In [ ]:
# Set four weeks for testing
train = df1.iloc[:436]
test = df1.iloc[436:]

### __Fit SARIMA(1,0,0)(2,0,0,7) model__
NOTE: To avoid a ValueError: non-invertible starting MA parameters found we're going to set enforce_invertibility to False.

In [ ]:
model = SARIMAX(train['total'],order=(1,0,0),seasonal_order=(2,0,0,7))

#https://stats.stackexchange.com/questions/50682/what-is-the-intuition-of-invertible-process-in-time-series

results = model.fit()
results.summary()

In [ ]:
# Obtain predicted values
start=len(train)
end=len(train)+len(test)-1
predictions = results.predict(start=start, end=end, dynamic=False).rename('SARIMA(1,0,0)(2,0,0,7) Predictions')

Passing dynamic=False means that forecasts at each point are generated using the full history up to that point (all lagged values).

More info here: https://www.statsmodels.org/stable/generated/statsmodels.tsa.arima_model.ARIMAResults.predict.html

In [ ]:
# Plot predictions against known values
title='Restaurant Visitors'
ylabel='Visitors per day' #xlalbel not required

ax = test['total'].plot(legend=True,figsize=(12,6),title=title)
predictions.plot(legend=True)
ax.autoscale(axis='x',tight=True)
ax.set(ylabel=ylabel)
for x in test.query('holiday==1').index: 
    ax.axvline(x=x, color='k', alpha = 0.3);

### __Model evaluation without exogenous variable__

In [ ]:
from statsmodels.tools.eval_measures import mse,rmse

mse_error = mse(test['total'], predictions)
rmse_error = rmse(test['total'], predictions)
mape_error = (sum(abs((test['total'] - predictions)\
                /test['total'])))*(100/len(test['total']))

print(f'SARIMA(1,0,0)(2,0,0,7) MSE: {mse_error:11.10}')
print(f'SARIMA(1,0,0)(2,0,0,7) RMSE: {rmse_error:11.10}')
print(f'SARIMA(1,0,0)(2,0,0,7) MAPE: {mape_error:11.10}')

## __Addition of exogenous variable__

In [ ]:
model = SARIMAX(train['total'],exog=train['holiday'],order=(1,0,0),seasonal_order=(2,0,0,7))
results = model.fit()
results.summary()

In [ ]:
# Obtain predicted values
start=len(train)
end=len(train)+len(test)-1
exog_forecast = test[['holiday']]  # requires two brackets to yield a shape of (35,1)
predictions = results.predict(start=start, end=end, exog=exog_forecast).rename('SARIMAX(1,0,0)(2,0,0,7) Predictions')

In [ ]:
# Plot predictions against known values
title='Restaurant Visitors'
ylabel='Visitors per day' #xlabel not needed

ax = test['total'].plot(legend=True,figsize=(12,6),title=title)
predictions.plot(legend=True)
ax.autoscale(axis='x',tight=True)
ax.set(ylabel=ylabel)
for x in test.query('holiday==1').index: 
    ax.axvline(x=x, color='k', alpha = 0.3);

We can see that the exogenous variable (holidays) had a positive impact on the forecast by raising predicted values at 3/17, 4/14, 4/16 and 4/17. What about evaluations?

### __Model evaluation including exogenous variable__

In [ ]:
# Print values from SARIMA above
print(f'SARIMA(1,0,0)(2,0,0,7) MSE Error: {mse_error:11.10}')
print(f'SARIMA(1,0,0)(2,0,0,7) RMSE Error: {rmse_error:11.10}')
print(f'SARIMA(1,0,0)(2,0,0,7) RMSE Error: {mape_error:11.10}')
print()

xmse_error = mse(test['total'], predictions)
xrmse_error = rmse(test['total'], predictions)
xmape_error = (sum(abs((test['total'] - predictions)\
                /test['total'])))*(100/len(test['total']))

# Print new SARIMAX values
print(f'SARIMAX(1,0,0)(2,0,0,7) MSE Error: {xmse_error:11.10}')
print(f'SARIMAX(1,0,0)(2,0,0,7) RMSE Error: {xrmse_error:11.10}')
print(f'SARIMAX(1,0,0)(2,0,0,7) RMSE Error: {xmape_error:11.10}')

__Our errors have improved by quite a bit. Isn#T that fance?__

If you want to get hints on how to include several exogenous variables - which is not completely intuitive, check out this link:
https://stackoverflow.com/questions/44212127/how-do-i-input-multiple-exogenous-variables-into-a-sarimax-model-in-statsmodel

### __Apply model to entire dataset to forecast into the future!__
We're going to forecast 39 days into the future, and use the additional holiday data

In [ ]:
model = SARIMAX(df1['total'],exog=df1['holiday'],order=(1,0,0),seasonal_order=(2,0,0,7),enforce_invertibility=False)
results = model.fit()
exog_forecast = df[478:][['holiday']]
fcast = results.predict(len(df1),len(df1)+38,exog=exog_forecast).rename('SARIMAX(1,0,0)(2,0,0,7) Forecast')

In [ ]:
# Plot the forecast alongside historical values
title='Restaurant Visitors'
ylabel='Visitors per day'
xlabel=''

ax = df1['total'].plot(legend=True,figsize=(16,6),title=title)
fcast.plot(legend=True)
ax.autoscale(axis='x',tight=True)
ax.set(xlabel=xlabel, ylabel=ylabel)
for x in df.query('holiday==1').index: 
    ax.axvline(x=x, color='k', alpha = 0.3);